## Generate Initial Positions
===============================================================================================

In [2]:
import numpy as np
import MDAnalysis as md
from MDAnalysis.analysis import align
import subprocess
import os
from MDAnalysis.analysis.leaflet import LeafletFinder

In [3]:
## Define paths to forcefield files and mdp files
ff_loc  = 'ff_itps/'
mdp_loc = 'mdps/'

## Define path to current directory
current_dir = '.'

## Define path to the directory where the stacked system was built
build_dir = '..'

In [4]:
def run_normal(num_slices, normal_dir, tail_bead, water_bead, umbrella_dist=0.1, umbrella_strength=0, pull_rate=0):
        ### Create folder to run normal
    if not os.path.exists(f'{normal_dir}'):
        os.makedirs(f'{normal_dir}')
        
    os.chdir(f'{normal_dir}')
    
    ###### Generate running .sh file
    os.system(f"echo '#!/bin/bash\n\
    source /projects/DAMM/gromacs/avx2256/gromacs-chain-coordinate/bin/GMXRC \n\
    #\n\
    # Set Chain coordinate parameters\n\
    #\n\
    export SLICE_COORD_N={num_slices}               # Number of slices. Important!\n\
                                          # The cylinder should be long enough, such that xi[ch]~0.2 in the state of a flat membrane\n\
    export SLICE_COORD_CYL_R=1.2          # Cylinder Radius\n\
    export SLICE_COORD_D=0.1              # Slice thickness (default value)\n\
    export SLICE_COORD_ZETA=0.75          # Degree to which slice is filled upon addition of the first atom (default value)\n\
    export SLICE_COORD_NST_OUTPUT=5000    # Output frequency on chain coord written by mdrun to stdout\n\
    export SLICE_COORD_FREEZE_CYLINDER=0  # Dot not freeze cylinder (default), allow cylinder to travel in the membrane plane\n\n\
    rm -f mdout.mdp topol.tpr\n\
    gmx grompp -c {build_dir}/eq1.gro -p {build_dir}/system_W.top -r {build_dir}/restraint_reference.gro -n {build_dir}/index.ndx -f normal.mdp\n\n\
    rm -f pullx.xvg pullf.xvg *cpt md.log ener.edr traj_comp.xtc confout.gro\n\
    gmx mdrun -v -nt 10'  > normal_membrane.sh")
    

    #### Edit .mdp file
    with open('common_files/md_template_1us.mdp', 'r') as f:
        ogfile= f.read()
    newfile=ogfile.replace('XXX', str(umbrella_dist))
    newfile=newfile.replace('YYY', str(umbrella_strength))
    newfile=newfile.replace('TTT', str(tail_bead))
    newfile=newfile.replace('WWW', str(water_bead))
    newfile=newfile.replace('RRR', str(pull_rate))
    with open('normal.mdp', 'w') as f:
        f.write(newfile)

    os.system('chmod +x normal_membrane.sh')
    #os.system('./normal_membrane.sh')
    os.chdir(f'{current_dir}')
    
    return print('All done')

In [5]:
def run_pulling(num_slices, tail_bead, water_bead, umbrella_dist=0.1, umbrella_strength=3000, pull_rate=1.5e-06):
    ###### Generate running .sh file
    os.system(f"echo '#!/bin/bash\n\
    source /projects/DAMM/gromacs/avx2256/gromacs-chain-coordinate/bin/GMXRC \n\
    #\n\
    # Set Chain coordinate parameters\n\
    #\n\
    export SLICE_COORD_N={num_slices}               # Number of slices. Important!\n\
                                          # The cylinder should be long enough, such that xi[ch]~0.2 in the state of a flat membrane\n\
    export SLICE_COORD_CYL_R=1.2          # Cylinder Radius\n\
    export SLICE_COORD_D=0.1              # Slice thickness (default value)\n\
    export SLICE_COORD_ZETA=0.75          # Degree to which slice is filled upon addition of the first atom (default value)\n\
    export SLICE_COORD_NST_OUTPUT=5000    # Output frequency on chain coord written by mdrun to stdout\n\
    export SLICE_COORD_FREEZE_CYLINDER=0  # Dot not freeze cylinder (default), allow cylinder to travel in the membrane plane\n\n\
    rm -f mdout.mdp topol.tpr\n\
    gmx grompp -c {build_dir}/eq1.gro -r {build_dir}/restraint_reference.gro -p {build_dir}/system_W.top -n {build_dir}/index.ndx -f pull.mdp\n\n\
    rm -f pullx.xvg pullf.xvg *cpt md.log ener.edr traj_comp.xtc confout.gro\n\
    gmx mdrun -v -nt 10 -pin on -pinstride 1 -pinoffset {pin_cores}'  > membrane_pulling.sh")
    
    #### Edit .mdp file
    with open('common_files/md_template_1us.mdp', 'r') as f:
        ogfile= f.read()
    newfile=ogfile.replace('XXX', str(umbrella_dist))
    newfile=newfile.replace('YYY', str(umbrella_strength))
    newfile=newfile.replace('TTT', str(tail_bead))
    newfile=newfile.replace('WWW', str(water_bead))
    newfile=newfile.replace('RRR', str(pull_rate))
    with open('pull.mdp', 'w') as f:
        f.write(newfile)
    
    os.system('chmod +x membrane_pulling.sh')
    #os.system('./membrane_pulling.sh')
    
    return print('All done')


#### Find normal
=====================================================================================

In [7]:
## Choose the number of slices
num_slices = 30

## Define the name of the directory to run the simulation where you determine the membrane normal
normal_dir = 'normal_sim'

## Manualy define a tail bead index from the lipids in contact with the middle water layer. 
tail_bead = 6526

## Manualy define a water bead index from the middle water compartment. 
water_bead = 8305

## Run
run_normal(num_slices, normal_dir, tail_bead, water_bead)

####  Pull two membranes together
============================================================

In [17]:
## Correct the number of slices if necessary
num_slices = 30

## Manualy define a tail bead index from the lipids in contact with the middle water layer. 
tail_bead = 6526

## Manualy define a water bead index from the middle water compartment. 
water_bead = 8305


for run, pin_cores in zip(['run_00', 'run_01', 'run_02', 'run_03', 'run_04'],[0, 10, 20, 30, 10]) : 
    os.chdir(current_dir)
    if os.path.isdir(run)==False:
        os.mkdir(run)
        os.chdir(run)
        run_pulling(num_slices, tail_bead, water_bead)
    elif os.path.isdir(run)==True:
        os.chdir(run)
        run_pulling(num_slices, tail_bead, water_bead)

All done
All done
All done
All done
All done
